In [1]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(verify_ssl_certs=False, model="GigaChat-2-Max")

In [26]:
from langchain.schema import HumanMessage

question = "N 63-ФЗ \"Об электронной подписи\""
llm.invoke([HumanMessage(content=question)]).content[0:200]

'Федеральный закон №\xa063-ФЗ от 06 апреля 2011 года — **«О внесении изменений в отдельные законодательные акты Российской Федерации в связи с принятием Федерального закона „Об электронной подписи”**».\n\nН'

In [2]:
list_of_files = [
'Положение Банка России от 30 января 2025 г N 851 П Об установлении обязательных .pdf',
'Приказ ФАПСИ от 13 июня 2001 г N 152 Об утверждении Инструкции об организации и .pdf',
'Приказ ФСБ РФ от 27 декабря 2011 г N 795 Об утверждении Требований к форме квали.pdf',
'Приказ ФСБ РФ от 27 декабря 2011 г N 796 Об утверждении Требований к средствам э.pdf',
'Приказ ФСБ РФ от 9 февраля 2005 г N 66 Об утверждении Положения о разработке про.pdf',
'Федеральный закон от 27 июля 2006 г N 152 ФЗ О персональных данных с изменениями.pdf',
'Федеральный закон от 6 апреля 2011 г N 63 ФЗ Об электронной подписи с изменениям.pdf']

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = PyPDFLoader("doc_files/" + list_of_files[0])
documents = loader.load()
# docs = [PyPDFLoader("doc_files/" + doc).load() for doc in list_of_files]

# flatted list
# documents = [item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,

)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 84


<Retrievers>

In [ ]:
class CustomRetriever()

In [ ]:
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings

from chromadb.config import Settings
from langchain_chroma import Chroma

embeddings = GigaChatEmbeddings(verify_ssl_certs=False)

vectorstore = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

retriever = vectorstore.as_retriever()

</Retrievers>

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Твоя задача — помогать пользователям получать данные из загруженных документов и отвечать на вопросы с использованием технологии извлечения знаний. Ты должен внимательно читать предоставленные документы и извлекать из них релевантную информацию для ответа на запросы пользователей.\n"
    "Основные правила работы:\n"
    "- Если вопрос связан с информацией из конкретного документа(ов), используй исключительно этот документ для формирования ответа.\n"
    "- Всегда указывай источник информации ссылкой на конкретный документ, из которого была извлечена информация. Формат цитирования: [Документ №X, страница Y].\n"
    # "Используй русскоязычные термины и формулировки там, где это возможно и уместно.\n"
    "- Строго придерживайся фактов, представленных в документах. Не добавляй лишнюю информацию от себя, если она отсутствует в документе.\n"
    "- Если какой-то факт невозможно подтвердить документами, обязательно сообщи об этом пользователю и предложи проверить другие источники.\n"
    "- Старайся давать четкий и понятный ответ, избегать длинных пояснений, если они не требуются.\n"
    "Пример правильного формата ответа:\n"
    "Вопрос: Какой размер налога на прибыль установлен в 2023 году?\n"
    "Ответ: В 2023 году налог на прибыль составляет 20%. [Документ 'пркиаз банка.pdf', стр. 5]\n"
    "Помни, твоя цель — точно передавать факты, содержащиеся в исходных документах, дополняя ответ источниками информации."
    
    # "Ты должен ответить на вопрос пользователя с использованием данных из банковских документов.\n"
    # "Вот части книги контекст для ответа:"
    
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [13]:
q = "Для каких целей нужно обеспечить использование усиленной неквалифицированной электронной подписи?"
# pure = llm([HumanMessage(content=q)]).content
# print(f"Pure: {pure}")

rag = rag_chain.invoke({"input": q})["answer"]
print(f"RAG: {rag}")

RAG: Усиленная неквалифицированная электронная подпись необходима для следующих целей:
- Подтверждение составления электронных сообщений уполномоченными лицами;
- Обеспечение целостности электронных сообщений;
- Защита информации с помощью средств криптографической защиты информации (СКЗИ).

Источник: [Документ 'Положение Банка России от 30 января 2025 г.', страницы 1–4].
